Ion Interaction
========
Ion interaction effects are important for correcting the mobility of ions based on the ionic strength of solution. These effects are laid out in the *Irreversible Processes in Electrohoresis*, by Onsager and Fuoss. 

These effects have prefactors that need to be converted to appropriate units for use. Here, I'm going to resolve the units on these prefactors, and resolve the differences between the paper and the STEEP implimentation.


In [1]:
# imports
from ionize import Aqueous
from math import sqrt, pi
import pint
ur = pint.UnitRegistry()
Q = ur.Quantity

In [2]:
# define values
temperature = Q(25, 'degC')
e = ur.elementary_charge
kb = ur.boltzmann_constant
dielectric = Aqueous.dielectric(temperature.magnitude)
viscosity = Aqueous.viscosity(temperature.magnitude) * ur.pascal * ur.second
Na = ur.avogadro_number
eps = ur.vacuum_permittivity

Robinson-Stokes
---------------

In [3]:
# STEEP: F*0.2297*z.*omega+31.410e-9
coefficient_1 = 0.2297 * ((temperature.to('degK'))**.5*dielectric)**3.
coefficient_2 = 31.410e-9 * viscosity * (temperature.to('degK')**.5*dielectric)

print(coefficient_1, coefficient_2)

568877142.2930058 kelvin ** 1.5 3.7840592709102225e-08 kelvin ** 0.5 * pascal * second


Onsager-Fuoss
------------
The Onsager-Fuoss constants A and B.

In [33]:
# Bhaga Paper
A = e**3 / 12 / pi 
A *= (Na / (dielectric * eps * kb * temperature.to('degK'))**3)**.5
print('[A] =', A.dimensionality)
print('A =', A.to('liter^.5/mol^.5'))

[A] = [length] ** 1.5 / [substance] ** 0.5
A = 0.5547184792860854 liter ** 0.5 / mole ** 0.5


In [35]:
B = e**2 / 6 / pi / viscosity 
B *= (Na / (dielectric * eps * kb * temperature.to('degK')))**.5
print('[B] =', B.dimensionality)
print('B =', B.to('m^2/V/s* liter^.5/mol^.5'))

[B] = [current] * [length] ** 1.5 * [time] ** 2 / [mass] / [substance] ** 0.5
B = 2.220870818024751e-08 liter ** 0.5 * meter ** 2 / mole ** 0.5 / second / volt


### Temperature-dependant values
Below are values that only include temperature independant versions. 

A = Ap / dielectric^1.5 / temperature^1.5

B = Bp x viscosity / dielectric^.5 / temperature^.5

In [7]:
Ap = e**3 / 12 / pi 
Ap *= (Na / (eps * kb)**3)**.5
print(Ap.dimensionality)
print(Ap.to('liter^.5*degK^1.5/mol^.5'))

[length] ** 1.5 * [temperature] ** 1.5 / [substance] ** 0.5
1980740.0746911524 kelvin ** 1.5 * liter ** 0.5 / mole ** 0.5


In [5]:
Bp = e**2 / 6 / pi 
Bp *= (Na / (eps * kb))**.5
print(Bp.dimensionality)
print(Bp.to('m^2/V/s* liter^.5/mol^.5 * Pa * s * degK**.5'))

[current] * [length] ** 0.5 * [temperature] ** 0.5 * [time] / [substance] ** 0.5
3.0225888785873703e-09 kelvin ** 0.5 * liter ** 0.5 * meter ** 2 * pascal / mole ** 0.5 / volt


In [8]:
# TODO: check steep and spresso implementation
# also check onsager-fuoss paper

Pitts
-----

In [26]:
D = (2 * e ** 2 * Na/ dielectric / eps / kb / temperature.to('degK'))**.5
print(D.dimensionality)
(D).to('m^.5/mol^.5')
(1/D / Q('(1e-7 mol/L)^.5')).to('um')

[length] ** 0.5 / [substance] ** 0.5


<Quantity(0.9610960030607913, 'micrometer')>

In [28]:
Dp = (2 * e ** 2 * Na / eps / kb)**.5
print(Dp.dimensionality)
Dp.to(())

[length] ** 0.5 * [temperature] ** 0.5 / [substance] ** 0.5
